In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import math
import random
import pickle as pkl
from datetime import datetime
from collections import Counter
#import matplotlib.pyplot as plt
#import seaborn as sns
import os
from os.path import join
import ujson as json
import re
from multiprocessing import Pool

import ujson as json
import pandas as pd
import pycld2
import pickle as pkl


In [2]:
os.chdir('/shared/3/projects/style-influence')

In [14]:
banned = pd.read_csv('/shared/3/projects/style-influence/data/subreddit-info/banned_subreddits_utc.txt')
print(banned.shape)
banned.head()

(60361, 3)


,Subreddit,UTC,Date_Banned
0,FTMmeetupGW,1514788393,2018-01-01
1,smallbuttstillsmokin,1514803636,2018-01-01
2,CryptoPets,1514808996,2018-01-01
3,tradenudepics,1514809858,2018-01-01
4,a2sd4f,1514840426,2018-01-01


In [26]:
formality = pd.DataFrame(pkl.load(open('data/dl-inferred-style-scores/all_scores.formality.pkl', 'rb')))
offensive = pd.DataFrame(pkl.load(open('data/dl-inferred-style-scores/all_scores.offensive.pkl', 'rb')))

In [27]:
formality['banned'] = formality.subreddit.isin(bsr)
offensive['banned'] = offensive.subreddit.isin(bsr)

In [28]:
Counter(formality['banned'])

Counter({False: 20061641, True: 178690})

In [29]:
formality.to_csv('working-dir/all_scores.formality.csv', index=None)
offensive.to_csv('working-dir/all_scores.offensive.csv', index=None)

In [24]:
au = set(formality[formality.banned].author)
len(au)

29653

In [23]:
len(au.intersection(set(formality[~formality.banned].author)))

13544

In [25]:
formality[(formality.author=='Saturatedseaunicorn') & (formality.banned)]

,row_id,subreddit,author,root_id,column_id,parent_id,depth,created_utc,score,banned
187,5354,hucows,Saturatedseaunicorn,fa60yhx,fa69js9,fa69ihr,19,1575822450,0.057,True
189,5356,hucows,Saturatedseaunicorn,fa60yhx,fa69c8d,fa69bfq,17,1575822363,0.213,True
191,5358,hucows,Saturatedseaunicorn,fa60yhx,fa68wo5,fa68lg0,15,1575822253,0.976,True
193,5360,hucows,Saturatedseaunicorn,fa60yhx,fa68hkr,fa68674,13,1575822144,0.712,True
195,5362,hucows,Saturatedseaunicorn,fa60yhx,fa66ysy,fa66uqq,11,1575821692,0.442,True
197,5364,hucows,Saturatedseaunicorn,fa60yhx,fa667nc,fa65v2r,9,1575821437,0.078,True
199,5366,hucows,Saturatedseaunicorn,fa60yhx,fa65omh,fa65ggw,7,1575821154,0.047,True
201,5368,hucows,Saturatedseaunicorn,fa60yhx,fa657ub,fa64sbe,5,1575821061,0.068,True
203,5370,hucows,Saturatedseaunicorn,fa60yhx,fa64fu1,fa63ytd,3,1575820829,0.108,True
205,5372,hucows,Saturatedseaunicorn,fa60yhx,fa63sjv,fa60yhx,1,1575820683,0.039,True


In [21]:
formality[(formality.author=='Saturatedseaunicorn') & (~formality.banned)]

,row_id,subreddit,author,root_id,column_id,parent_id,depth,created_utc,score,banned
40517,901690,littlespace,Saturatedseaunicorn,fa1army,fa63qik,fa5tu58,17,1575820667,0.057,False
40519,901692,littlespace,Saturatedseaunicorn,fa1army,fa5t6zn,fa5numt,15,1575818128,0.013,False
40521,901694,littlespace,Saturatedseaunicorn,fa1army,fa5n082,fa5mqvj,13,1575816479,0.047,False
40523,901696,littlespace,Saturatedseaunicorn,fa1army,fa5mifu,fa5lw4x,11,1575816311,0.098,False
40525,901698,littlespace,Saturatedseaunicorn,fa1army,fa5jyhx,fa5jgas,9,1575815729,0.008,False
40527,901700,littlespace,Saturatedseaunicorn,fa1army,fa5j4c5,fa5fkz8,7,1575815541,0.006,False
40529,901702,littlespace,Saturatedseaunicorn,fa1army,fa5erua,fa5ek1c,5,1575814401,0.031,False
40531,901704,littlespace,Saturatedseaunicorn,fa1army,fa5ed9w,fa5e28t,3,1575814242,0.009,False
40533,901706,littlespace,Saturatedseaunicorn,fa1army,fa5d31j,fa1army,1,1575813966,0.011,False
5458004,504687,littlespace,Saturatedseaunicorn,fa65utc,fa69h10,fa69eu4,11,1575822418,0.046,False


In [6]:
sorted([f for f in os.listdir('data/conversation-turns-tokenized-aba') if f.endswith('.csv')])

['RC_2019-06.user_subreddit.csv',
 'RC_2019-09.user_subreddit.csv',
 'RC_2019-10.user_subreddit.csv',
 'RC_2019-11.user_subreddit.csv',
 'RC_2019-12.user_subreddit.csv',
 'RC_2020-01.user_subreddit.csv',
 'RC_2020-02.user_subreddit.csv',
 'RC_2020-03.user_subreddit.csv',
 'RC_2020-04.user_subreddit.csv',
 'RC_2020-05.user_subreddit.csv',
 'RC_2020-06.user_subreddit.csv',
 'RC_2020-07.user_subreddit.csv',
 'RC_2020-08.user_subreddit.csv',
 'RC_2020-09.user_subreddit.csv',
 'RC_2020-10.user_subreddit.csv',
 'RC_2020-11.user_subreddit.csv',
 'RC_2020-12.user_subreddit.csv',
 'RC_2021-01.user_subreddit.csv',
 'RC_2021-02.user_subreddit.csv',
 'RC_2021-03.user_subreddit.csv',
 'RC_2021-04.user_subreddit.csv',
 'RC_2021-05.user_subreddit.csv',
 'RC_2021-06.user_subreddit.csv',
 'RC_2021-07.user_subreddit.csv',
 'RC_2021-09.user_subreddit.csv',
 'RC_2021-10.user_subreddit.csv',
 'RC_2021-11.user_subreddit.csv',
 'RC_2021-12.user_subreddit.csv',
 'RC_2022-01.user_subreddit.csv',
 'RC_2022-02.u